# Import librarys

In [1]:
from keras.models import load_model
from keras.metrics import MeanSquaredError
from sklearn.base import BaseEstimator
from keras.models import Model
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import keras
from keras.callbacks import History
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import keras
from scikeras.wrappers import KerasRegressor
from keras.layers import Input
from keras.optimizers import RMSprop
import yfinance as yf
from matplotlib.backends.backend_pdf import PdfPages
import pickle
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dropout, Dense
from tensorflow.keras.losses import MeanSquaredError
from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV

# CSV einlesen

In [2]:
def download_stock_data(stock_symbols, start_date, end_date, output_file):
    with open(output_file, 'w') as f:
        # Write the column headings
        f.write('Date,Open,High,Low,Close,Volume\n')
        
        for symbol in stock_symbols:
            # Load the share data for every symbol and the defined time period
            stock_data = yf.download(symbol, start=start_date, end=end_date)
            stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Volume']]  # Auswahl der gewünschten Spalten
            stock_data.to_csv(f, header=False)  # Schreiben der Daten in die Datei


In [3]:
# Define time period and symbol
stock_symbols = ['ADS.DE']
start_date = '2014-01-01'
end_date = '2018-12-31'
output_file = 'stock_data.csv'

# Load and store the share data
download_stock_data(stock_symbols, start_date, end_date, output_file)

# Search and delete empty rows in the CSV file
with open(output_file, 'r') as file:
    lines = file.readlines()

# Filter empty rows
lines = [line.strip() for line in lines if line.strip()]

# Overwrite the file with the cleaned rows
with open(output_file, 'w') as file:
    file.write('\n'.join(lines))

[*********************100%%**********************]  1 of 1 completed


In [4]:
# load CSV
df = pd.read_csv('stock_data.csv')

# transform date to index
df.set_index('Date', inplace=True)

# Show first rows
print(df.head())
print(len(df))

                 Open       High        Low      Close  Volume
Date                                                          
2014-01-02  92.879997  93.220001  91.139999  91.400002  801548
2014-01-03  91.389999  91.580002  90.470001  91.220001  613843
2014-01-06  91.209999  91.489998  90.910004  91.169998  475794
2014-01-07  91.059998  91.330002  90.230003  91.070000  650288
2014-01-08  91.360001  91.510002  90.389999  90.610001  593016
1268


In [5]:
# Check, if there are any missing numbers
missing_values_before = df.isnull().values.any()

# Treat missing values with the mean of the previous and the following row
for column in df.columns:
    missing_values = df[column].isnull()
    df.loc[missing_values, column] = (df[column].shift() + df[column].shift(-1)) / 2

# Check, if there are missing values after the Treatment
missing_values_after = df.isnull().values.any()

# Output of the missing values before and after the Treatment
print("Fehlende Werte vor der Behandlung gefunden:", missing_values_before)
print("Fehlende Werte nach der Behandlung gefunden:", missing_values_after)

Fehlende Werte vor der Behandlung gefunden: False
Fehlende Werte nach der Behandlung gefunden: False


In [6]:
# show first rows
print(df.head())

                 Open       High        Low      Close  Volume
Date                                                          
2014-01-02  92.879997  93.220001  91.139999  91.400002  801548
2014-01-03  91.389999  91.580002  90.470001  91.220001  613843
2014-01-06  91.209999  91.489998  90.910004  91.169998  475794
2014-01-07  91.059998  91.330002  90.230003  91.070000  650288
2014-01-08  91.360001  91.510002  90.389999  90.610001  593016


# Data normalization

In [7]:
# Copy the data frame and remove the column "date"
nf = df.copy()

# Remove the index name
nf.index.name = None

# Normalize the data, except the "date" (index)
scaler = MinMaxScaler()
nf_normalized = scaler.fit_transform(nf)

# Create a new data frame with the normalized data and the original index
nf = pd.DataFrame(nf_normalized, columns=nf.columns, index=nf.index)

# # Show the normalized data
print(nf.head())

# Show the length of the data frame
print("Länge des Datensatzes:", len(nf))

                Open      High       Low     Close    Volume
2014-01-02  0.238584  0.237473  0.234846  0.231386  0.066880
2014-01-03  0.229487  0.227451  0.230727  0.230276  0.051218
2014-01-06  0.228388  0.226900  0.233432  0.229967  0.039700
2014-01-07  0.227473  0.225923  0.229251  0.229350  0.054259
2014-01-08  0.229304  0.227023  0.230235  0.226513  0.049481
Länge des Datensatzes: 1268


In [8]:
# save scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# read predicting sequence

In [9]:
# Define time period and output file
stock_symbols = ['BNR.VI']
start_date = '2018-10-01'
end_date = '2018-12-31'
output_file = 'stock_data_pre.csv'

# Load and store the share data
download_stock_data(stock_symbols, start_date, end_date, output_file)

# Search and delete the empty rows in the CSV file
with open(output_file, 'r') as file:
    lines = file.readlines()

# Filter the empty rows
lines = [line.strip() for line in lines if line.strip()]

# Overwrite the file with the cleaned rows
with open(output_file, 'w') as file:
    file.write('\n'.join(lines))

[*********************100%%**********************]  1 of 1 completed


In [10]:
# load CSV
dfPre = pd.read_csv('stock_data_pre.csv')

# Drop empty rows
dfPre.dropna(inplace=True)

# transform date to index
dfPre.set_index('Date', inplace=True)

# Check the number of rows
num_rows = dfPre.shape[0]

# If more than 60 rows, keep the last 60 rows
if num_rows > 60:
    dfPre = dfPre.tail(60)

# Show first rows
print(dfPre.head())

                 Open       High        Low      Close  Volume
Date                                                          
2018-10-03  51.119999  51.119999  51.119999  51.119999       0
2018-10-04  51.119999  51.119999  51.119999  51.119999       0
2018-10-05  51.119999  51.119999  51.119999  51.119999       0
2018-10-08  51.119999  51.119999  51.119999  51.119999       0
2018-10-09  51.119999  51.119999  51.119999  51.119999       0


In [11]:
# Check, if there are missing values before the treatment
missing_values_beforePre = dfPre.isnull().values.any()

# Treat missing values for all columns with the mean of the previous and following row
for column in df.columns:
    missing_valuesPre = dfPre[column].isnull()
    dfPre.loc[missing_valuesPre, column] = (dfPre[column].shift() + dfPre[column].shift(-1)) / 2

# Check, if there are missing values after the treatment
missing_values_afterPre = dfPre.isnull().values.any()

# Output of the missing values before and after the treatment
print("Fehlende Werte vor der Behandlung gefunden:", missing_values_beforePre)
print("Fehlende Werte nach der Behandlung gefunden:", missing_values_afterPre)

# show lenth of data set
print(len(dfPre))

Fehlende Werte vor der Behandlung gefunden: False
Fehlende Werte nach der Behandlung gefunden: False
60


# Data normalization prediction data

In [12]:
# Copy the data frame and remove the column "date"
nfPre = dfPre.copy()

# Remove the index name
nfPre.index.name = None

# Normalize the data, except the "date" (Index)
nfPre_normalized = scaler.fit_transform(nfPre)

# Create a data frame with the normalized data and the original index
nfPre = pd.DataFrame(nfPre_normalized, columns=nfPre.columns, index=nfPre.index)

# Convert DataFrame to NumPy array
nfPre = nfPre.to_numpy()

# Show length of the data frame
print("Länge des Datensatzes:", len(nfPre))


Länge des Datensatzes: 60


# Create Sequences

In [13]:
# Function to create sequenences for LSTM model
def create_sequences(data_input, seq_length):
    data = data_input
    X, y = [], []
    for i in range(len(data) - seq_length - 20):
        X.append(data[i:i+seq_length])
        y.append(data.iloc[i+seq_length+20]) # acess to Close-Value of the 20th row
    return np.array(X), np.array(y)

In [14]:
# create sequence
sequence_length = 60
X, y = create_sequences(nf, sequence_length)

# Split the data in Training, Test and validation

In [15]:
# Split data in training and test (80% training, 20% test)
train_data, test_data = train_test_split(nf, test_size=0.2, shuffle=False)

# Split data in training and validation (80% training, 20% validation)
train_data, val_data = train_test_split(train_data, test_size=0.2, shuffle=False)

# Split the train data in sequences
X_train, y_train = create_sequences(train_data, sequence_length)

# Split the validation data in sequences
X_val, y_val = create_sequences(val_data, sequence_length)

# Split the test data in sequences
X_test, y_test = create_sequences(test_data, sequence_length)

# show training, validation and test sets
print("Trainingsdaten:", len(train_data))
print("Validierungsdaten:", len(val_data))
print("Testdaten:", len(test_data))

Trainingsdaten: 811
Validierungsdaten: 203
Testdaten: 254


In [16]:
# Define train_size, val_size and test_size
train_size = len(train_data)
val_size = len(val_data)
test_size = len(test_data)

# Extract features and target variables for training, validation, and test sets
X_train, y_train = X[:train_size], y[:train_size]
X_val, y_val = X[train_size:train_size+val_size], y[train_size:train_size+val_size]
X_test, y_test = X[train_size+val_size:], y[train_size+val_size:]

print(X_train.shape)
print(y_train.shape)

(811, 60, 5)
(811, 5)


In [17]:
# Convert the train, val and prediction data into tensors
X_train_tensor = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val_tensor = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val_tensor = tf.convert_to_tensor(y_val, dtype=tf.float32)

print(type(X_train_tensor))
print(type(y_train_tensor))

<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>


# Define model

In [18]:
param_dist = {
    'units': [25, 50, 100, 300, 400, 500, 800],
    'dropout_rate': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
    'batch_size': [16, 32, 64, 128, 256],
    'epochs': [10, 20, 30, 40, 50, 80, 100],
    'optimizer': ['adam', 'rmsprop'],
    'learning_rate': [0.001, 0.003, 0.005, 0.008, 0.01],
}

In [19]:
class LSTMModel(BaseEstimator):
    def __init__(self, batch_size=32, optimizer='adam', units=50, dropout_rate=0.2, epochs=10, learning_rate=0.001):
        self.batch_size = batch_size
        self.optimizer = optimizer
        self.units = units
        self.dropout_rate = dropout_rate
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.model = self.create_lstm_model()  # Hier das Modell erstellen
        
    def create_lstm_model(self):
        if self.optimizer == 'adam':
            optimizer = Adam(learning_rate=self.learning_rate)
        elif self.optimizer == 'rmsprop':
            optimizer = RMSprop(learning_rate=self.learning_rate)
        else:
            raise ValueError("Unrecognized optimizer name.")
        input_layer = Input(shape=(X_train.shape[1], X_train.shape[2]))
        lstm_layer1 = LSTM(units=self.units, return_sequences=True)(input_layer)
        dropout_layer1 = Dropout(self.dropout_rate)(lstm_layer1)
        lstm_layer2 = LSTM(units=self.units, return_sequences=False)(dropout_layer1)
        dropout_layer2 = Dropout(self.dropout_rate)(lstm_layer2)
        output_layer = Dense(units=5)(dropout_layer2)
    
        model = Model(inputs=input_layer, outputs=output_layer)
    
        model.compile(optimizer=optimizer, loss='mse', metrics=[MeanSquaredError()])
    
        return model
        
    def fit(self, X, y):
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size)
        
    def predict(self, X):
        return self.model.predict(X)

In [20]:
lstm_model = LSTMModel()  # Create a instance of the lstm model
random_search = RandomizedSearchCV(estimator=lstm_model,
                                   param_distributions=param_dist,
                                   n_iter=10,
                                   cv=3,
                                   scoring='neg_mean_squared_error',
                                   verbose=2,
                                   random_state=42)

# Train the model with the best hyperparameters
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 0.1247 - mean_squared_error: 0.1247
Epoch 2/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0234 - mean_squared_error: 0.0234
Epoch 3/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0166 - mean_squared_error: 0.0166
Epoch 4/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0113 - mean_squared_error: 0.0113
Epoch 5/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0096 - mean_squared_error: 0.0096
Epoch 6/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0087 - mean_squared_error: 0.0087
Epoch 7/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0086 - mean_squared_error: 0.0086
Epoch 8/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0091 - mean_squared_error: 0.0091
Epoch 9/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0079 - mean_squared_error: 0.0079
Epoch 10/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0079 - mean_squared_error: 0.0079
Epoch 11

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 65/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 66/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0042 - mean_squared_error: 0.0042
Epoch 67/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0040 - mean_squared_error: 0.0040
Epoch 68/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 69/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 70/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 71/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0042 - mean_squared_error: 0.0042
Epoch 72/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 73/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 74/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0049 - mean_

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0057 - mean_squared_error: 0.0057
Epoch 47/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0068 - mean_squared_error: 0.0068
Epoch 48/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 49/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 50/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 51/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0054 - mean_squared_error: 0.0054
Epoch 52/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 53/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 54/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0058 - mean_squared_error: 0.0058
Epoch 55/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 56/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0054 - mean_

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 29/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0023 - mean_squared_error: 0.0023
Epoch 30/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0026 - mean_squared_error: 0.0026
Epoch 31/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 32/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0029 - mean_squared_error: 0.0029
Epoch 33/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 0.0025 - mean_squared_error: 0.0025
Epoch 34/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0024 - mean_squared_error: 0.0024
Epoch 35/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 36/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 37/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0027 - mean_squared_error: 0.0027
Epoch 38/80
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0019 - mean_

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 11/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 12/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0058 - mean_squared_error: 0.0058
Epoch 13/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0062 - mean_squared_error: 0.0062
Epoch 14/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0059 - mean_squared_error: 0.0059
Epoch 15/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0061 - mean_squared_error: 0.0061
Epoch 16/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0065 - mean_squared_error: 0.0065
Epoch 17/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 18/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0060 - mean_squared_error: 0.0060
Epoch 19/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 20/80
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step 

...

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 90/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0023 - mean_squared_error: 0.0023
Epoch 91/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 92/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 93/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 94/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 95/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0023 - mean_squared_error: 0.0023
Epoch 96/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 97/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 98/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 99/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 51/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0033 - mean_squared_error: 0.0033
Epoch 52/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 53/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 54/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 55/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 56/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 57/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 58/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 59/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0040 - mean_squared_error: 0.0040
Epoch 60/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0021 - mean_squared_error: 0.0021
Epoch 12/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0024 - mean_squared_error: 0.0024
Epoch 13/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 14/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0028 - mean_squared_error: 0.0028
Epoch 15/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 16/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 17/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0020 - mean_squared_error: 0.0020
Epoch 18/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0025 - mean_squared_error: 0.0025
Epoch 19/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 20/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0019 - mean_squared_error: 0.0019
Epoch 21/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 

34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0022 - mean_squared_error: 0.0022
Epoch 74/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 75/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 76/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 77/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 78/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0014 - mean_squared_error: 0.0014
Epoch 79/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0013 - mean_squared_error: 0.0013
Epoch 80/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0018 - mean_squared_error: 0.0018
Epoch 81/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0017 - mean_squared_error: 0.0017
Epoch 82/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0015 - mean_squared_error: 0.0015
Epoch 83/100
34/34 ━━━━━━━━━━━━━━━━━━━━ 1s 

RandomizedSearchCV(cv=3, estimator=LSTMModel(),
                   param_distributions={'batch_size': [16, 32, 64, 128, 256],
                                        'dropout_rate': [0.05, 0.1, 0.2, 0.3,
                                                         0.4, 0.5],
                                        'epochs': [10, 20, 30, 40, 50, 80, 100],
                                        'learning_rate': [0.001, 0.003, 0.005,
                                                          0.008, 0.01],
                                        'optimizer': ['adam', 'rmsprop'],
                                        'units': [25, 50, 100, 300, 400, 500,
                                                  800]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [21]:
# Output the best hyperparameter
print("Beste Hyperparameter:", random_search.best_params_)

Beste Hyperparameter: {'units': 25, 'optimizer': 'adam', 'learning_rate': 0.003, 'epochs': 10, 'dropout_rate': 0.5, 'batch_size': 128}
